# Import Dependencies

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import math
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable

import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as data
import torchvision
from torchvision import transforms

from keras.layers import Input, Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten
from keras.models import Model
import matplotlib.pyplot as plt
import pickle

/Users/michellezhao/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/michellezhao/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/michellezhao/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/michellezhao/.local/lib/p

In [4]:
BATCH_SIZE = 64 
print("batch size", BATCH_SIZE)
LEARNING_RATE = 0.003
TRAIN_DATA_PATH = "categories_fruits_big/train/"
TEST_DATA_PATH = "categories_fruits_big/test/"
# TRANSFORM_IMG = transforms.Compose([
#     transforms.Resize(256),
#     transforms.CenterCrop(256),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                          std=[0.229, 0.224, 0.225] )
#     ])

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# transform_test = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
# ])


transform_test = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


train_data = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=transform_train)
train_data_loader = data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True,  num_workers=4)
test_data = torchvision.datasets.ImageFolder(root=TEST_DATA_PATH, transform=transform_test)
test_data_loader  = data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4) 

trainset = train_data
trainloader = train_data_loader
testset = test_data
testloader = test_data_loader

batch size 64


In [5]:
print("Number of train samples: ", len(train_data))
print("Number of test samples: ", len(test_data))
print("Detected Classes are: ", train_data.class_to_idx) # classes are detected by folder structure


Number of train samples:  23789
Number of test samples:  7674
Detected Classes are:  {'apple': 0, 'banana': 1, 'onion': 2, 'pear': 3, 'pepper': 4, 'pineapple': 5, 'plum': 6, 'potato': 7, 'strawberry': 8, 'tomato': 9}


In [6]:
print("num classes = ", len(train_data.class_to_idx))

num classes =  10


# Create Model

In [7]:
class Net3(nn.Module):
    def __init__(self):
        super(Net3, self).__init__()
        self.conv1 = nn.Conv2d(3, 28, 3,padding=1)
        self.bn1 = nn.BatchNorm2d(28)
        self.conv2 = nn.Conv2d(28, 35, 3,padding=1)
        self.bn2 = nn.BatchNorm2d(35)
        self.pool = nn.MaxPool2d(2, 2)
        self.drop1 = nn.Dropout(p=0.2)
        self.conv3 = nn.Conv2d(35, 16, 3,padding=1)
        self.bn3 = nn.BatchNorm2d(16)
        
        self.fc1 = nn.Linear(16 * 8 * 8, 100)
        self.drop2 = nn.Dropout(p=0.3)
        self.fc2 = nn.Linear(100, 30)
        self.bn4 = nn.BatchNorm1d(30)
        self.fc3 = nn.Linear(30, 10)

    def forward(self, x):
        x = self.bn1(F.relu(self.conv1(x)))
        x = self.drop1(x)
        x = self.bn2(F.relu(self.conv2(x)))
        x = self.pool(x)
        x = self.bn3(F.relu(self.conv3(x)))
        x = self.pool(x)
        x = x.view(-1, 16 * 8 * 8)
        x = self.drop1(x)
        x = F.relu(self.fc1(x))
        x = self.drop2(x)
        x = self.bn4(F.relu(self.fc2(x)))
        x = self.fc3(x)
    
        return F.log_softmax(x, dim=0)


# Simulating Active Learning

# Active Model: Minimum Largest Difference

# Plot with Test Error

In [8]:
def compute_test_acc(model, testloader):
    # Test the model
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        count = 0
        for images, labels in testloader:
            count += 1
            images = Variable(images)
            labels = Variable(labels)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return (correct / total)

def compute_train_acc(model, trainloader):
    # Test the model
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        count = 0
        for images, labels in trainloader:
            count += 1
            images = Variable(images)
            labels = Variable(labels)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return (correct / total)

In [ ]:
# Find optimal_batch
def uncertainty_metric_max_min_difference_selection(model, trainloader):
    batch_evaluation = []
    for i, (images, labels) in enumerate(trainloader):
        outputs = model(images)
        batch_differences = []
        for j in range(len(outputs)):
            min_prob = min(outputs[j])
            max_prob = max(outputs[j])
            diff = float(max_prob - min_prob)
            batch_differences.append(diff)
        batch_mean = np.mean(batch_differences)
        batch_evaluation.append(batch_mean)
    batch_evaluation = np.array(batch_evaluation)
    k = 20
#     largest = list((-batch_evaluation).argsort()[:k])
    smallest = list(np.argsort(batch_evaluation)[:k])
    return smallest

In [ ]:
loss_list = []
acc_list = []
train_err_list = []
test_err_list = []

for trial in range(3):
    active_model = Net3()
    free_params = sum(p.numel() for p in active_model.parameters() if p.requires_grad)
    print(free_params)

    trainstep = 125
    # Loss and optimizer

    criterion = nn.NLLLoss() #You can modify the loss function
    optimizer = optim.SGD(active_model.parameters(), lr=0.005, momentum=0.9, weight_decay=8e-4) #You can change the optimizer

    # Train the model

    total_step = len(trainloader)
    print(total_step)
    trial_loss_list = []
    trial_acc_list = []

    trial_train_err_list = []
    trial_test_err_list = []

    num_epochs = 200

    for epoch in range(num_epochs):
        print("epoch: ", epoch)
        total = 0
        correct = 0
        randomly_selected = uncertainty_metric_max_min_difference_selection(active_model, trainloader)
        for i, (images, labels) in enumerate(trainloader):
            if i not in randomly_selected:
                continue

    #         images = images.cuda(async=True)
    #         labels = labels.cuda(async=True)

            images = Variable(images)
            labels = Variable(labels)
            # Run the forward pass
    #         print("running forward pass....")
            outputs = active_model(images)

            loss = criterion(outputs, labels)
            trial_loss_list.append(loss.item())

            # Backprop 
    #         print("backpropagating......")
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

             # Track the accuracy
            total = labels.size(0) + total
            _, predicted = torch.max(outputs.data, 1)
            correct = (predicted == labels).sum().item() + correct
            trial_acc_list.append(correct / total)


            if (i + 1) % trainstep == 0:
                w = torch.nn.utils.parameters_to_vector(active_model.parameters())
                print(w)
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                      .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                              (correct / total) * 100))
        trial_train_err_list.append(compute_train_acc(active_model, trainloader))
        trial_test_err_list.append(compute_test_acc(active_model, testloader))
        if (total == correct):
            break 

#     loss_list.append(np.array(trial_loss_list))
#     acc_list.append(trial_acc_list)
    train_err_list.append(trial_train_err_list)
    test_err_list.append(trial_test_err_list)
            
print('Finished Training') 

# loss_list = np.array(loss_list)
# acc_list = np.array(acc_list)
train_err_list= np.array(train_err_list)
test_err_list=np.array(test_err_list)

# loss_list = np.mean(loss_list, axis=0)
# acc_list = np.mean(acc_list, axis=0)
train_err_list= np.mean(train_err_list, axis=0)
test_err_list=np.mean(test_err_list, axis=0)




In [ ]:
# testing = []
# testing.append([1,2,3])
# testing.append([4,2,3])
# testing = np.array(testing)
# print(np.mean(testing, axis=0))

In [ ]:
# ... after training, save your model 
model_filename = 'res_test2/active_model3_minimum.pt'
torch.save(active_model.state_dict(), model_filename)

# .. to load your previously training model:
# model2 = Net3()
# model2.load_state_dict(torch.load(model_filename))
# model2.eval()

In [ ]:
active_train_err_list = train_err_list
active_test_err_list = test_err_list
active_acc_list1 = acc_list



In [ ]:
pickle_out = open("res_test2/test1_largest_margin_minimum_training_accuracy.pickle","wb")
pickle.dump(active_train_err_list, pickle_out)
pickle_out.close()


pickle_out = open("res_test2/test1_largest_margin_minimum_testing_accuracy.pickle","wb")
pickle.dump(active_test_err_list, pickle_out)
pickle_out.close()

# pickle_out = open("test1_random_testing_accuracy.pickle","wb")
# pickle.dump(new_rand_test_err_list, pickle_out)
# pickle_out.close()

# Random Model

In [ ]:
# random_model2 = Net3()

In [ ]:
rand_loss_list = []
rand_acc_list = []
rand_train_err_list = []
rand_test_err_list = []

for trial in range(3):
    random_model2 = Net3()
    free_params = sum(p.numel() for p in random_model2.parameters() if p.requires_grad)

    trainstep = 125
    # Loss and optimizer

    criterion = nn.NLLLoss() #You can modify the loss function
    optimizer = optim.SGD(random_model2.parameters(), lr=0.005, momentum=0.9, weight_decay=8e-4) #You can change the optimizer

    # Train the model

    total_step = len(trainloader)
    print(total_step)
    trial_rand_loss_list = []
    trial_rand_acc_list = []

    trial_rand_train_err_list = []
    trial_rand_test_err_list = []

    num_epochs = 200

    for epoch in range(num_epochs):
        print("epoch: ", epoch)
        total = 0
        correct = 0
        randomly_selected = np.random.choice(range(len(list(trainloader))), size=20)
        for i, (images, labels) in enumerate(trainloader):
            if i not in randomly_selected:
                continue

    #         images = images.cuda(async=True)
    #         labels = labels.cuda(async=True)

            images = Variable(images)
            labels = Variable(labels)
            # Run the forward pass
    #         print("running forward pass....")
            outputs = random_model2(images)

            loss = criterion(outputs, labels)
            rand_loss_list.append(loss.item())

            # Backprop 
    #         print("backpropagating......")
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

             # Track the accuracy
            total = labels.size(0) + total
            _, predicted = torch.max(outputs.data, 1)
            correct = (predicted == labels).sum().item() + correct
            rand_acc_list.append(correct / total)


            if (i + 1) % trainstep == 0:
                w = torch.nn.utils.parameters_to_vector(random_model2.parameters())
                print(w)
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                      .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                              (correct / total) * 100))
        trial_rand_train_err_list.append(compute_train_acc(random_model2, trainloader))
        trial_rand_test_err_list.append(compute_test_acc(random_model2, testloader))
        if (total == correct):
            break 

    rand_loss_list.append(trial_rand_loss_list)
    rand_acc_list.append(trial_rand_acc_list)
    rand_train_err_list.append(trial_rand_train_err_list)
    rand_test_err_list.append(trial_rand_test_err_list)
            
print('Finished Training') 

# rand_loss_list = np.array(rand_loss_list)
# rand_acc_list = np.array(rand_acc_list)
rand_train_err_list= np.array(rand_train_err_list)
rand_test_err_list=np.array(rand_test_err_list)

# rand_loss_list = np.mean(rand_loss_list, axis=0)
# rand_acc_list = np.mean(rand_acc_list, axis=0)
rand_train_err_list= np.mean(rand_train_err_list, axis=0)
rand_test_err_list=np.mean(rand_test_err_list, axis=0)





## Compare Largest Margin to Random

In [ ]:
new_active_train_err_list = [0]
new_active_train_err_list.extend(active_train_err_list)
plt.plot(new_active_train_err_list)
# plt.plot(active_test_err_list)
new_rand_train_err_list = [0]
new_rand_train_err_list.extend(rand_train_err_list)
plt.plot(new_rand_train_err_list)
# plt.plot(active_test_err_list)
plt.legend(['active', 'random'])
plt.title("Active Training Set Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Training Set Accuracy")
plt.savefig("res_test2/test3_4_acc1.png")
plt.show()

pickle_out = open("res_test2/test1_largest_margin_minimum2_training_accuracy.pickle","wb")
pickle.dump(new_active_train_err_list, pickle_out)
pickle_out.close()

pickle_out = open("res_test2/test1_random2_training_accuracy.pickle","wb")
pickle.dump(new_rand_train_err_list, pickle_out)
pickle_out.close()

In [ ]:
new_active_test_err_list = [0]
new_active_test_err_list.extend(active_test_err_list)

# plt.plot(new_second_train_err_list)
# plt.plot(active_test_err_list)
new_rand_test_err_list = [0]
new_rand_test_err_list.extend(rand_test_err_list)

plt.plot(new_active_test_err_list)
# plt.plot(active_test_err_list)
plt.plot(new_rand_test_err_list)
# plt.plot(active_test_err_list)
plt.legend(['active', 'random'])
plt.title("Active Test Set Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Test Set Accuracy")
plt.savefig("res_test2/test3_4_acc2.png")
plt.show()

pickle_out = open("res_test2/test2_largest_margin_minimum2_testing_accuracy.pickle","wb")
pickle.dump(new_active_test_err_list, pickle_out)
pickle_out.close()

pickle_out = open("res_test2/test2_random2_testing_accuracy.pickle","wb")
pickle.dump(new_rand_test_err_list, pickle_out)
pickle_out.close()

In [ ]:
# ... after training, save your model 
model_filename = 'res_test2/random_model2.pt'
torch.save(random_model2.state_dict(), model_filename)

# .. to load your previously training model:
# model2 = Net3()
# model2.load_state_dict(torch.load(model_filename))
# model2.eval()

In [ ]:
plt.plot(active_train_err_list)
# plt.plot(active_test_err_list)
plt.plot(rand_train_err_list)
# plt.plot(active_test_err_list)
plt.legend(['active', 'random'])
plt.title("Active Training Set Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Training Set Accuracy")
plt.savefig("res_test2/test2_acc1_min.png")
plt.show()

In [ ]:
plt.plot(active_test_err_list)
# plt.plot(active_test_err_list)
plt.plot(rand_test_err_list)
# plt.plot(active_test_err_list)
plt.legend(['active', 'random'])
plt.title("Active Training Set Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Training Set Accuracy")
plt.savefig("res_test2/test2_acc2_min.png")
plt.show()

# Second Diff

In [ ]:
# second_model1 = Net3()

In [26]:
# Find optimal_batch
def uncertainty_metric_max_second_difference_selection(model, trainloader):
    batch_evaluation = []
    for i, (images, labels) in enumerate(trainloader):
        outputs = model(images)
        batch_differences = []
        for j in range(len(outputs)):
            
#             min_prob = min(outputs[j])
            vals = outputs[j].data.numpy()
            top_two = vals[np.argsort(vals)[-2:]]
            
            diff = abs(float(top_two[0] - top_two[1]))
            batch_differences.append(diff)
        batch_mean = np.mean(batch_differences)
        batch_evaluation.append(batch_mean)
    batch_evaluation = np.array(batch_evaluation)
    k = 20
#     largest = list((-batch_evaluation).argsort()[:k])
    smallest = list(np.argsort(batch_evaluation)[:k])
    return smallest

In [27]:
second_loss_list = []
second_acc_list = []
second_train_err_list = []
second_test_err_list = []

for trial in range(3):
    second_model1 = Net3()
    free_params = sum(p.numel() for p in second_model1.parameters() if p.requires_grad)
# print(free_params)

    trainstep = 125
    # Loss and optimizer

    criterion = nn.NLLLoss() #You can modify the loss function
    optimizer = optim.SGD(second_model1.parameters(), lr=0.005, momentum=0.9, weight_decay=8e-4) #You can change the optimizer

    # Train the model

    total_step = len(trainloader)
    print(total_step)
    trial_second_loss_list = []
    trial_second_acc_list = []

    trial_second_train_err_list = []
    trial_second_test_err_list = []

    num_epochs = 200

    for epoch in range(num_epochs):
        print("epoch: ", epoch)
        total = 0
        correct = 0
        randomly_selected = uncertainty_metric_max_second_difference_selection(second_model1, trainloader)
        for i, (images, labels) in enumerate(trainloader):
            if i not in randomly_selected:
                continue

    #         images = images.cuda(async=True)
    #         labels = labels.cuda(async=True)

            images = Variable(images)
            labels = Variable(labels)
            # Run the forward pass
    #         print("running forward pass....")
            outputs = second_model1(images)

            loss = criterion(outputs, labels)
            trial_second_loss_list.append(loss.item())

            # Backprop 
    #         print("backpropagating......")
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

             # Track the accuracy
            total = labels.size(0) + total
            _, predicted = torch.max(outputs.data, 1)
            correct = (predicted == labels).sum().item() + correct
            trial_second_acc_list.append(correct / total)


            if (i + 1) % trainstep == 0:
                w = torch.nn.utils.parameters_to_vector(second_model1.parameters())
                print(w)
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                      .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                              (correct / total) * 100))
        trial_second_train_err_list.append(compute_train_acc(second_model1, trainloader))
        trial_second_test_err_list.append(compute_test_acc(second_model1, testloader))
        if (total == correct):
            break 

    second_loss_list.append(trial_second_loss_list)
    second_acc_list.append(trial_second_acc_list)
    second_train_err_list.append(trial_second_train_err_list)
    second_test_err_list.append(trial_second_test_err_list)
            
print('Finished Training') 

second_loss_list = np.array(second_loss_list)
second_acc_list = np.array(second_acc_list)
second_train_err_list= np.array(second_train_err_list)
second_test_err_list=np.array(second_test_err_list)

second_loss_list = np.mean(second_loss_list, axis=0)
second_acc_list = np.mean(second_acc_list, axis=0)
second_train_err_list= np.mean(second_train_err_list, axis=0)
second_test_err_list=np.mean(second_test_err_list, axis=0)







372
epoch:  0
epoch:  1
epoch:  2
epoch:  3
epoch:  4
epoch:  5
epoch:  6
epoch:  7
epoch:  8
epoch:  9
epoch:  10
epoch:  11
epoch:  12
epoch:  13
epoch:  14
epoch:  15
epoch:  16
epoch:  17
epoch:  18
epoch:  19


Process Process-1671:
Process Process-1672:
Process Process-1670:
Process Process-1669:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/michellezhao/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/michellezhao/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/michellezhao/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/michellezhao/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/michellezhao/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/michellezhao/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File

KeyboardInterrupt: 

In [ ]:
# ... after training, save your model 
model_filename = 'res_test2/second_model3_small.pt'
torch.save(second_model1.state_dict(), model_filename)

# .. to load your previously training model:
# model2 = Net3()
# model2.load_state_dict(torch.load(model_filename))
# model2.eval()

In [ ]:
new_second_train_err_list = [0]
new_second_train_err_list.extend(second_train_err_list)
plt.plot(new_second_train_err_list)
# plt.plot(active_test_err_list)
new_rand_train_err_list = [0]
new_rand_train_err_list.extend(rand_train_err_list)
plt.plot(new_rand_train_err_list)
# plt.plot(active_test_err_list)
plt.legend(['active', 'random'])
plt.title("Active Training Set Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Training Set Accuracy")
plt.savefig("res_test2/test2_4_acc1.png")
plt.show()

pickle_out = open("res_test2/test1_smallest_margin_minimum_training_accuracy.pickle","wb")
pickle.dump(new_second_train_err_list, pickle_out)
pickle_out.close()

pickle_out = open("res_test2/test1_random_training_accuracy.pickle","wb")
pickle.dump(new_rand_train_err_list, pickle_out)
pickle_out.close()

In [ ]:
new_second_test_err_list = [0]
new_second_test_err_list.extend(second_test_err_list)

# plt.plot(new_second_train_err_list)
# plt.plot(active_test_err_list)
new_rand_test_err_list = [0]
new_rand_test_err_list.extend(rand_test_err_list)

plt.plot(new_second_test_err_list)
# plt.plot(active_test_err_list)
plt.plot(new_rand_test_err_list)
# plt.plot(active_test_err_list)
plt.legend(['active', 'random'])
plt.title("Active Test Set Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Test Set Accuracy")
plt.savefig("res_test2/test2_4_acc2.png")
plt.show()

pickle_out = open("res_test2/test1_smallest_margin_minimum_testing_accuracy.pickle","wb")
pickle.dump(new_second_test_err_list, pickle_out)
pickle_out.close()

pickle_out = open("res_test2/test1_random_testing_accuracy.pickle","wb")
pickle.dump(new_rand_test_err_list, pickle_out)
pickle_out.close()

In [ ]:
plt.plot(second_test_err_list)
plt.plot(active_test_err_list)
# plt.plot(rand_test_err_list)
# plt.plot(active_test_err_list)
plt.legend(['smallest margin', 'largest margin', 'random'])
plt.title("Active Test Set Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Test Set Accuracy")
plt.savefig("res_test2/test2_acc2.png")
plt.show()

# Compare Active, Second, Random

In [ ]:
new_active_train_err_list = [0]
new_active_train_err_list.extend(active_train_err_list)
plt.plot(new_active_train_err_list)

new_second_train_err_list = [0]
new_second_train_err_list.extend(second_train_err_list)
plt.plot(new_second_train_err_list)
# plt.plot(active_test_err_list)
new_rand_train_err_list = [0]
new_rand_train_err_list.extend(rand_train_err_list)
plt.plot(new_rand_train_err_list)
# plt.plot(active_test_err_list)
plt.legend(['largest margin', 'smallest margin', 'random'])
plt.title("Active Training Set Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Training Set Accuracy")
plt.savefig("res_test2/comparisontest2_4_acc1.png")
plt.show()


pickle_out = open("res_test2/test2_largest_margin_minimum_training_accuracy.pickle","wb")
pickle.dump(new_active_train_err_list, pickle_out)
pickle_out.close()


pickle_out = open("res_test2/test2_smallest_margin_minimum_training_accuracy.pickle","wb")
pickle.dump(new_second_train_err_list, pickle_out)
pickle_out.close()

pickle_out = open("res_test2/test2_random_training_accuracy.pickle","wb")
pickle.dump(new_rand_train_err_list, pickle_out)
pickle_out.close()

In [ ]:
new_active_test_err_list = [0]
new_active_test_err_list.extend(active_test_err_list)

new_second_test_err_list = [0]
new_second_test_err_list.extend(second_test_err_list)

# plt.plot(new_second_train_err_list)
# plt.plot(active_test_err_list)
new_rand_test_err_list = [0]
new_rand_test_err_list.extend(rand_test_err_list)

plt.plot(new_active_test_err_list)

plt.plot(new_second_test_err_list)
# plt.plot(active_test_err_list)
plt.plot(new_rand_test_err_list)
# plt.plot(active_test_err_list)
plt.legend(['largest margin', 'smallest margin', 'random'])
plt.title("Active Test Set Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Test Set Accuracy")
plt.savefig("res_test2/comparisontest2_4_acc2.png")
plt.show()

pickle_out = open("res_test2/test2_largest_margin_minimum_testing_accuracy.pickle","wb")
pickle.dump(new_active_test_err_list, pickle_out)
pickle_out.close()

pickle_out = open("res_test2/test2_smallest_margin_minimum_testing_accuracy.pickle","wb")
pickle.dump(new_second_test_err_list, pickle_out)
pickle_out.close()

pickle_out = open("res_test2/test2_random_testing_accuracy.pickle","wb")
pickle.dump(new_rand_test_err_list, pickle_out)
pickle_out.close()

# Entropy Reduction

In [ ]:
# entropy_model1 = Net3()

In [ ]:
# Find optimal_batch
def uncertainty_metric_entropy_selection(model, trainloader):
    batch_evaluation = []
    for i, (images, labels) in enumerate(trainloader):
        outputs = model(images)
        batch_differences = []
        for j in range(len(outputs)):
            total = 0
            output_probs = outputs[j].data.numpy()
            for item in output_probs:
                entropy = -1* item * np.log(item)
                total += entropy
            batch_differences.append(total)
            
        batch_mean = np.mean(batch_differences)
        batch_evaluation.append(batch_mean)
    batch_evaluation = np.array(batch_evaluation)
    k = 20
    largest = list((-batch_evaluation).argsort()[:k])
#     smallest = list(np.argsort(batch_evaluation)[:k])
    return largest

In [ ]:
entropy_loss_list = []
entropy_acc_list = []
entropy_train_err_list = []
entropy_test_err_list = []

for trial in range(3):
    entropy_model1 = Net3()
    free_params = sum(p.numel() for p in entropy_model1.parameters() if p.requires_grad)
    print(free_params)

    trainstep = 125
    # Loss and optimizer

    criterion = nn.NLLLoss() #You can modify the loss function
    optimizer = optim.SGD(entropy_model1.parameters(), lr=0.005, momentum=0.9, weight_decay=8e-4) #You can change the optimizer

    # Train the model

    total_step = len(trainloader)
    print(total_step)
    trial_entropy_loss_list = []
    trial_entropy_acc_list = []

    trial_entropy_train_err_list = []
    trial_entropy_test_err_list = []

    num_epochs = 200

    for epoch in range(num_epochs):
        print("epoch: ", epoch)
        total = 0
        correct = 0
        randomly_selected = uncertainty_metric_entropy_selection(entropy_model1, trainloader)
        for i, (images, labels) in enumerate(trainloader):
            if i not in randomly_selected:
                continue

    #         images = images.cuda(async=True)
    #         labels = labels.cuda(async=True)

            images = Variable(images)
            labels = Variable(labels)
            # Run the forward pass
    #         print("running forward pass....")
            outputs = entropy_model1(images)

            loss = criterion(outputs, labels)
            trial_entropy_loss_list.append(loss.item())

            # Backprop 
    #         print("backpropagating......")
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

             # Track the accuracy
            total = labels.size(0) + total
            _, predicted = torch.max(outputs.data, 1)
            correct = (predicted == labels).sum().item() + correct
            trial_entropy_acc_list.append(correct / total)


            if (i + 1) % trainstep == 0:
                w = torch.nn.utils.parameters_to_vector(entropy_model1.parameters())
                print(w)
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                      .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                              (correct / total) * 100))
        trial_entropy_train_err_list.append(compute_train_acc(entropy_model1, trainloader))
        trial_entropy_test_err_list.append(compute_test_acc(entropy_model1, testloader))
        if (total == correct):
            break 

    entropy_loss_list.append(trial_entropy_loss_list)
    entropy_acc_list.append(trial_entropy_acc_list)
    entropy_train_err_list.append(trial_entropy_train_err_list)
    entropy_test_err_list.append(trial_entropy_test_err_list)
            
print('Finished Training') 

entropy_loss_list = np.array(entropy_loss_list)
entropy_acc_list = np.array(entropy_acc_list)
entropy_train_err_list= np.array(entropy_train_err_list)
entropy_test_err_list=np.array(entropy_test_err_list)

entropy_loss_list = np.mean(entropy_loss_list, axis=0)
entropy_acc_list = np.mean(entropy_acc_list, axis=0)
entropy_train_err_list= np.mean(entropy_train_err_list, axis=0)
entropy_test_err_list=np.mean(entropy_test_err_list, axis=0)






In [ ]:
# ... after training, save your model 
model_filename = 'res_test2/entropy_model1.pt'
torch.save(entropy_model1.state_dict(), model_filename)

# .. to load your previously training model:
# model2 = Net3()
# model2.load_state_dict(torch.load(model_filename))
# model2.eval()

In [ ]:
plt.plot(entropy_train_err_list)
# plt.plot(active_test_err_list)
plt.plot(rand_train_err_list)
# plt.plot(active_test_err_list)
plt.legend(['active', 'random'])
plt.title("Active Training Set Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Training Set Accuracy")
plt.savefig("res_test2/test3_acc1.png")
plt.show()

In [ ]:
plt.plot(entropy_test_err_list)
# plt.plot(active_test_err_list)
plt.plot(rand_test_err_list)
# plt.plot(active_test_err_list)
plt.legend(['active', 'random'])
plt.title("Active Test Set Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Test Set Accuracy")
plt.savefig("res_test2/test3_acc2.png")
plt.show()

In [ ]:
new_entropy_train_err_list = [0]
new_entropy_train_err_list.extend(entropy_train_err_list)
plt.plot(new_entropy_train_err_list)
# plt.plot(active_test_err_list)
new_rand_train_err_list = [0]
new_rand_train_err_list.extend(rand_train_err_list)
plt.plot(new_rand_train_err_list)
# plt.plot(active_test_err_list)
plt.legend(['active', 'random'])
plt.title("Active Training Set Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Training Set Accuracy")
plt.savefig("res_test2/test3_4_acc1.png")
plt.show()

pickle_out = open("res_test2/test1_entropy_maxmimum_training_accuracy.pickle","wb")
pickle.dump(new_entropy_train_err_list, pickle_out)
pickle_out.close()

# pickle_out = open("test1_random_training_accuracy.pickle","wb")
# pickle.dump(new_rand_train_err_list, pickle_out)
# pickle_out.close()

In [ ]:
new_entropy_test_err_list = [0]
new_entropy_test_err_list.extend(entropy_test_err_list)

# plt.plot(new_second_train_err_list)
# plt.plot(active_test_err_list)
new_rand_test_err_list = [0]
new_rand_test_err_list.extend(rand_test_err_list)

plt.plot(new_entropy_test_err_list)
# plt.plot(active_test_err_list)
plt.plot(new_rand_test_err_list)
# plt.plot(active_test_err_list)
plt.legend(['active', 'random'])
plt.title("Active Test Set Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Test Set Accuracy")
plt.savefig("res_test2/test3_4_acc2.png")
plt.show()

pickle_out = open("res_test2/test1_entropy_maximum_testing_accuracy.pickle","wb")
pickle.dump(new_entropy_test_err_list, pickle_out)
pickle_out.close()

# pickle_out = open("test1_random_testing_accuracy.pickle","wb")
# pickle.dump(new_rand_test_err_list, pickle_out)
# pickle_out.close()

In [ ]:
new_active_train_err_list = [0]
new_active_train_err_list.extend(active_train_err_list)
plt.plot(new_active_train_err_list)

new_second_train_err_list = [0]
new_second_train_err_list.extend(second_train_err_list)
plt.plot(new_second_train_err_list)
# plt.plot(active_test_err_list)
new_rand_train_err_list = [0]
new_rand_train_err_list.extend(rand_train_err_list)
plt.plot(new_rand_train_err_list)

new_entropy_train_err_list = [0]
new_entropy_train_err_list.extend(entropy_train_err_list)
plt.plot(new_entropy_train_err_list)


# plt.plot(active_test_err_list)
plt.legend(['largest margin', 'smallest margin','entropy', 'random'])
plt.title("Active Training Set Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Training Set Accuracy")
plt.savefig("res_test2/comparisontest2_6_acc1.png")
plt.show()


In [ ]:
new_active_test_err_list = [0]
new_active_test_err_list.extend(active_test_err_list)

new_second_test_err_list = [0]
new_second_test_err_list.extend(second_test_err_list)

# plt.plot(new_second_train_err_list)
# plt.plot(active_test_err_list)
new_rand_test_err_list = [0]
new_rand_test_err_list.extend(rand_test_err_list)

plt.plot(new_active_test_err_list)

plt.plot(new_second_test_err_list)
# plt.plot(active_test_err_list)
plt.plot(new_rand_test_err_list)

new_entropy_test_err_list = [0]
new_entropy_test_err_list.extend(entropy_test_err_list)
plt.plot(new_entropy_test_err_list)


# plt.plot(active_test_err_list)
plt.legend(['largest margin', 'smallest margin','entropy',  'random'])
plt.title("Active Test Set Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Test Set Accuracy")
plt.savefig("res_test2/comparisontest2_6_acc2.png")
plt.show()

# Least Confident

In [ ]:
# lc_model1 = Net3()

In [ ]:
# Find optimal_batch
def uncertainty_metric_lc_selection(model, trainloader):
    batch_evaluation = []
    for i, (images, labels) in enumerate(trainloader):
        outputs = model(images)
        batch_differences = []
        for j in range(len(outputs)):
            max_prob = max(outputs[j])
            diff = float(1 - max_prob)
            batch_differences.append(diff)
        batch_mean = np.sum(batch_differences)
        batch_evaluation.append(batch_mean)
    batch_evaluation = np.array(batch_evaluation)
    k = 20
    return list((-batch_evaluation).argsort()[:k])

In [ ]:
lc_loss_list = []
lc_acc_list = []
lc_train_err_list = []
lc_test_err_list = []

for trial in range(3):
    lc_model1 = Net3()
    free_params = sum(p.numel() for p in lc_model1.parameters() if p.requires_grad)
    print(free_params)

    trainstep = 125
    # Loss and optimizer

    criterion = nn.NLLLoss() #You can modify the loss function
    optimizer = optim.SGD(lc_model1.parameters(), lr=0.005, momentum=0.9, weight_decay=8e-4) #You can change the optimizer

    # Train the model

    total_step = len(trainloader)
    print(total_step)
    trial_lc_loss_list = []
    trial_lc_acc_list = []

    trial_lc_train_err_list = []
    trial_lc_test_err_list = []

    num_epochs = 200

    for epoch in range(num_epochs):
        print("epoch: ", epoch)
        total = 0
        correct = 0
        randomly_selected = uncertainty_metric_lc_selection(lc_model1, trainloader)
        for i, (images, labels) in enumerate(trainloader):
            if i not in randomly_selected:
                continue

    #         images = images.cuda(async=True)
    #         labels = labels.cuda(async=True)

            images = Variable(images)
            labels = Variable(labels)
            # Run the forward pass
    #         print("running forward pass....")
            outputs = lc_model1(images)

            loss = criterion(outputs, labels)
            trial_lc_loss_list.append(loss.item())

            # Backprop 
    #         print("backpropagating......")
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

             # Track the accuracy
            total = labels.size(0) + total
            _, predicted = torch.max(outputs.data, 1)
            correct = (predicted == labels).sum().item() + correct
            trial_lc_acc_list.append(correct / total)


            if (i + 1) % trainstep == 0:
                w = torch.nn.utils.parameters_to_vector(lc_model1.parameters())
                print(w)
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                      .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                              (correct / total) * 100))
        trial_lc_train_err_list.append(compute_train_acc(lc_model1, trainloader))
        trial_lc_test_err_list.append(compute_test_acc(lc_model1, testloader))
        if (total == correct):
            break 

    lc_loss_list.append(trial_lc_loss_list)
    lc_acc_list.append(trial_lc_acc_list)
    lc_train_err_list.append(trial_lc_train_err_list)
    lc_test_err_list.append(trial_lc_test_err_list)
            
print('Finished Training') 

lc_loss_list = np.array(lc_loss_list)
lc_acc_list = np.array(lc_acc_list)
lc_train_err_list= np.array(lc_train_err_list)
lc_test_err_list=np.array(lc_test_err_list)

lc_loss_list = np.mean(lc_loss_list, axis=0)
lc_acc_list = np.mean(lc_acc_list, axis=0)
lc_train_err_list= np.mean(lc_train_err_list, axis=0)
lc_test_err_list=np.mean(lc_test_err_list, axis=0)
 




In [ ]:
# ... after training, save your model 
model_filename = 'res_test2/lc_model1.pt'
torch.save(lc_model1.state_dict(), model_filename)

# .. to load your previously training model:
# model2 = Net3()
# model2.load_state_dict(torch.load(model_filename))
# model2.eval()

In [ ]:
plt.plot(lc_train_err_list)
# plt.plot(active_test_err_list)
plt.plot(rand_train_err_list)
# plt.plot(active_test_err_list)
plt.legend(['active', 'random'])
plt.title("Active Training Set Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Training Set Accuracy")
plt.savefig("res_test2/test4_acc1.png")
plt.show()

In [ ]:
plt.plot(lc_test_err_list)
# plt.plot(active_test_err_list)
plt.plot(rand_test_err_list)
# plt.plot(active_test_err_list)
plt.legend(['active', 'random'])
plt.title("Active Test Set Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Test Set Accuracy")
plt.savefig("res_test2/test4_acc2.png")
plt.show()

In [ ]:
new_lc_train_err_list = [0]
new_lc_train_err_list.extend(lc_train_err_list)
plt.plot(new_lc_train_err_list)
# plt.plot(active_test_err_list)
new_rand_train_err_list = [0]
new_rand_train_err_list.extend(rand_train_err_list)
plt.plot(new_rand_train_err_list)
# plt.plot(active_test_err_list)
plt.legend(['active', 'random'])
plt.title("Active Training Set Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Training Set Accuracy")
plt.savefig("res_test2/test4_4_acc1.png")
plt.show()

pickle_out = open("res_test2/test1_lc_maxmimum_training_accuracy.pickle","wb")
pickle.dump(new_entropy_train_err_list, pickle_out)
pickle_out.close()

# pickle_out = open("test1_random_training_accuracy.pickle","wb")
# pickle.dump(new_rand_train_err_list, pickle_out)
# pickle_out.close()

In [ ]:
new_lc_test_err_list = [0]
new_lc_test_err_list.extend(lc_test_err_list)

# plt.plot(new_second_train_err_list)
# plt.plot(active_test_err_list)
new_rand_test_err_list = [0]
new_rand_test_err_list.extend(rand_test_err_list)

plt.plot(new_lc_test_err_list)
# plt.plot(active_test_err_list)
plt.plot(new_rand_test_err_list)
# plt.plot(active_test_err_list)
plt.legend(['active', 'random'])
plt.title("Active Test Set Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Test Set Accuracy")
plt.savefig("res_test2/test4_4_acc2.png")
plt.show()

pickle_out = open("res_test2/test1_lc_maximum_testing_accuracy.pickle","wb")
pickle.dump(new_entropy_test_err_list, pickle_out)
pickle_out.close()

# pickle_out = open("test1_random_testing_accuracy.pickle","wb")
# pickle.dump(new_rand_test_err_list, pickle_out)
# pickle_out.close()

# Compare All

In [ ]:
new_active_train_err_list = [0]
new_active_train_err_list.extend(active_train_err_list)
plt.plot(new_active_train_err_list)

new_second_train_err_list = [0]
new_second_train_err_list.extend(second_train_err_list)
plt.plot(new_second_train_err_list)
# plt.plot(active_test_err_list)
new_rand_train_err_list = [0]
new_rand_train_err_list.extend(rand_train_err_list)
plt.plot(new_rand_train_err_list)

new_entropy_train_err_list = [0]
new_entropy_train_err_list.extend(entropy_train_err_list)
plt.plot(new_entropy_train_err_list)

new_lc_train_err_list = [0]
new_lc_train_err_list.extend(lc_train_err_list)
plt.plot(new_lc_train_err_list)

# plt.plot(active_test_err_list)
plt.legend(['largest margin', 'smallest margin','entropy', 'least-confidence', 'random'])
plt.title("Active Training Set Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Training Set Accuracy")
plt.savefig("res_test2/comparisontest2_5_acc1.png")
plt.show()


In [ ]:
new_active_test_err_list = [0]
new_active_test_err_list.extend(active_test_err_list)

new_second_test_err_list = [0]
new_second_test_err_list.extend(second_test_err_list)

# plt.plot(new_second_train_err_list)
# plt.plot(active_test_err_list)
new_rand_test_err_list = [0]
new_rand_test_err_list.extend(rand_test_err_list)

plt.plot(new_active_test_err_list)

plt.plot(new_second_test_err_list)
# plt.plot(active_test_err_list)
plt.plot(new_rand_test_err_list)

new_entropy_test_err_list = [0]
new_entropy_test_err_list.extend(entropy_test_err_list)
plt.plot(new_entropy_test_err_list)

new_lc_test_err_list = [0]
new_lc_test_err_list.extend(lc_test_err_list)
plt.plot(new_lc_test_err_list)

# plt.plot(active_test_err_list)
plt.legend(['largest margin', 'smallest margin','entropy', 'least-confidence', 'random'])
plt.title("Active Test Set Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Test Set Accuracy")
plt.savefig("res_test2/comparisontest2_5_acc2.png")
plt.show()